## Script for generating temperature response to country-level emissions preturbation by way of FaIR
#### Mustafa Zahid, 05/27/22

In [5]:
#Importing main packages as well as packages to generate the distributions 
# from which we will pull the sets of parameters
from scipy.stats import lognorm, norm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# Import date class from datetime module
from datetime import datetime
import os

In [6]:
today = datetime.now()
today = today.strftime('%Y%m%d')
path = '/Users/' + os.getlogin() + '/BurkeLab Dropbox/Projects/loss_damage/FaIR'

In [7]:
if not os.path.exists(path + "/" + today):
    os.mkdir(path + "/" + today)

### Part I: Generating climatic parameters (following Ashwin et al., 2021)

In [8]:
# Writing the functions to generate distributions for our climate parameters (inputs in FaIR model)
def lognorm_from_percentiles(x1, p1, x2, p2):
    """ Return a log-normal distribuion X parametrized by:
    
            P(X < p1) = x1
            P(X < p2) = x2
    """
    x1 = np.log(x1)
    x2 = np.log(x2)
    p1ppf = norm.ppf(p1)
    p2ppf = norm.ppf(p2)
    
    scale = (x2 - x1) / (p2ppf - p1ppf)
    mean = ((x1 * p2ppf) - (x2 * p1ppf)) / (p2ppf - p1ppf)
    
    return np.random.lognormal(mean, scale, 1000)

In [ ]:
# Writing the functions to generate distributions for our climate parameters (inputs in FaIR model)
def norm_from_percentiles(x1, p1, x2, p2):
    """ Return a normal distribuion X parametrized by:
    
            P(X < p1) = x1
            P(X < p2) = x2
    """
    p1ppf = norm.ppf(p1)
    p2ppf = norm.ppf(p2)

    location = ((x1 * p2ppf) - (x2 * p1ppf)) / (p2ppf - p1ppf)
    scale = (x2 - x1) / (p2ppf - p1ppf)

    return np.random.normal(location, scale, 1000)

In [9]:
# Now iterating over the parameters to pull from the distributions
tcr = []
rwf = []
tau = []
d2 = []
ecs = []
idi = []
for i in range(1,101):
        idi.append(i)
        tcr.append(np.random.choice(lognorm_from_percentiles(1, 0.17, 2.5, 0.83)))
        rwf.append(np.random.choice(norm_from_percentiles(0.45, 0.25, 0.75, 0.75)))
        tau.append(np.random.choice(np.random.normal(4.03, 1.79, 10000)))
        d2.append(np.random.choice(lognorm_from_percentiles(1.6, 0.05, 8.4, 0.95))) 

NameError: name 'norm_from_percentiles' is not defined

In [ ]:
# Now generate a dataframe and add the pulled parameters together
df = pd.DataFrame()
df["rwfnum"] = rwf
df["taunum"] = tau
df["tcrnum"] = tcr
df["d2num"] = d2

In [ ]:
# Following Ashwin et al., we restrict our parameters 
#tau
df = df[df["taunum"] > 0] 
df = df[df["taunum"] < (2*4.03)]

#rwf
df = df[df["rwfnum"] < 1]
df = df[df["rwfnum"] > np.percentile(df["rwfnum"],6)]

#ecs
df["ecs"] = df["tcrnum"] / df["rwfnum"]

In [ ]:
#generate an ID for each of the sets of parameters
df["idnum"] = range(len(df))

In [10]:
df = pd.read_csv(path + "/fair_params/fair_parameter_set_052323.csv")

### Part II: Running the FaIR model by iterating over emissions preturbation scenarios

In [11]:
# read in the needed libraries 
import numpy as np
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
from fair import *
#%matplotlib inline

In [14]:
# read the emissions country-year-level data
data = pd.read_csv(path + '/emissions_data/long_term_emissions_data_2300_v2021.csv')  
data1 = pd.read_csv(path + '/emissions_data/country_lvl_emms_v2022_2100.csv')

In [ ]:
# read the emissions country-year-level data
data = pd.read_csv(path + 'emissions_data/long_term_emissions_data_2300_v2021.csv')  
data1 = pd.read_csv(path + 'emissions_data/country_lvl_consump_emms_v2022.csv")

In [ ]:
data1 = pd.read_csv("country_lvl_prod_emms_v2022.csv")

In [15]:
data = data[data["Year"] > 1989]

In [16]:
data1 = data1[data1["Year"] > 1989]

In [17]:
data = data[data["Year"] < 2101]

In [18]:
data1 = data1[data1["Year"] < 2101]

In [19]:
 # The following function returns an empty dataframe in the correct format for use in FaIR.
test_emissions = return_empty_emissions(df_to_copy=False, 
                                        start_year=0, 
                                        end_year=111, 
                                        timestep=1, 
                                        scen_names=['Test'], 
                                        gases_in = ['carbon_dioxide'])

In [20]:
  # create an empty forcing dataframe compatible with test_emissions:
test_forcing = return_empty_forcing(test_emissions)

    # Note that the scenario names in the emissions and forcing dataframe must be identical for 
    #the model to run: FaIR assumes each emissions scenario corresponds directly to a single forcing scenario.
    
    # Generate a default parameter dataframe:
test_gas_parameters = get_gas_parameter_defaults()
test_thermal_parameters = get_thermal_parameter_defaults()

    # This is what the gas parameter dataframe looks like:
test_gas_parameters.head()
test_gas_parameters = test_gas_parameters.reindex(test_emissions.columns.levels[1],
                                                  axis=1,
                                                  level=1)

In [23]:
list_of_isos = pd.unique(data1["ISO3"])

In [ ]:
list_of_isos[200:217]

In [21]:
usa = list("USA")

In [ ]:
i = "USA"
data_iso = data1[data1["ISO3"] == i]
data_iso = data_iso[["Year", "Total"]]
data_iso.rename(columns = {'Total':'Total_iso'}, inplace = True)
data_new = pd.merge(data, data_iso, on = "Year")
data_new["Total"] = data_new["Total"] - data_new["Total_iso"]
data_new = data_new[["Year", "Total"]]
data_new.rename(columns = {'Total':'carbon_dioxide'}, inplace = True) #change name to match emissions data
data_new.rename(columns = {'Year':'year'}, inplace = True) #change name to match emissions data
data_new["year"] = pd.to_numeric(data_new["year"]) 
# since the carbon emissions are on MtCO2, we need to convert to GtC
data_new['carbon_dioxide'] = (data_new['carbon_dioxide'] / 1000) / 3.67 
data_new = data_new[data_new["year"] >= 1980].reset_index() # only keep 1980 and forward
data_new['year'] = data_new['year'] - (min(data_new['year']) - 1) 

In [ ]:
usa

In [24]:
list_dfs = []
for m in range(1,len(df)): 
    for i in list_of_isos[204:217]:
        # shut off emissions from country i
        data_iso = data1[data1["ISO3"] == i]
        data_iso = data_iso[["Year", "Total"]]
        data_iso.rename(columns = {'Total':'Total_iso'}, inplace = True)
        data_new = pd.merge(data, data_iso, on = "Year")
        data_new["Total"] = data_new["Total"] - data_new["Total_iso"]
        data_new = data_new[["Year", "Total"]]
        data_new.rename(columns = {'Total':'carbon_dioxide'}, inplace = True) #change name to match emissions data
        data_new.rename(columns = {'Year':'year'}, inplace = True) #change name to match emissions data
        data_new["year"] = pd.to_numeric(data_new["year"]) 
        # since the carbon emissions are on MtCO2, we need to convert to GtC
        data_new['carbon_dioxide'] = (data_new['carbon_dioxide'] / 1000) / 3.67 
        data_new = data_new[data_new["year"] >= 1980].reset_index() # only keep 1980 and forward
        data_new['year'] = data_new['year'] - (min(data_new['year']) - 1) 
        #feed updated emissions to test_emissions
        test_emissions[("Test", "carbon_dioxide")] = data_new['carbon_dioxide']
        
        # therefore use central estimates for d1-3, q1; and set q2, q3 to return "correct" TCR/ECS values
        df1 = df[df['idnum'] == m] 
        test_gas_parameters[("default", "carbon_dioxide")][7] = list(df1["taunum"])[0]
        d1 = 0.903
        d2 = list(df1["d2num"])[0]
        d3 = 355
        q1 = 0.180
        rwf = list(df1["rwfnum"])[0]

        TCR = list(df1["tcrnum"])[0]
        ECS = TCR / rwf
        F_2x = 3.759

        v1 = (1-(d1/69.66) * (1-np.exp(-69.66/d1)) )
        v2 = (1-(d2/69.66) * (1-np.exp(-69.66/d2)) )
        v3 = (1-(d3/69.66) * (1-np.exp(-69.66/d3)) )

        q3 = (((TCR/F_2x) - q1*(v1-v2) - (ECS/F_2x)*v2) / (v3-v2))
        q2 = (ECS/F_2x - q1 -  q3)

        test_thermal_parameters["default",2][1] = q2
        test_thermal_parameters["default",3][1] = q3

            #test_thermal_parameters = pd.DataFrame([[d1,d2,d3],[q1,q2,q3]],
             #                                      index=['d','q'],
              #                                     columns=pd.MultiIndex.from_product([['default'],
                      #                                                                 [1,2,3]]))
        pulse_run = run_FaIR(emissions_in=test_emissions,
                             forcing_in=test_forcing,
                             gas_parameters=test_gas_parameters,
                             thermal_parameters=test_thermal_parameters)
        temp_response = pulse_run['T']
        #(temp_response.temp_response).apply(lambda x: float(x))   
            #temp_response["temp_response"] = float(temp_response["temp_response"]) 
            #temp_response.to_csv('temp_response/tempresponse' + "_" + str(k) + "_" + str(iso) + '.csv') 
        temp_response["loop"] = str(m) + "_loop_" + i   
        list_dfs.append(temp_response)
        print(i)
        print(m)
        # save dataset 
pd.concat(list_dfs).to_csv(path + "/" + today + "/fair_temp_resp_k90_hist_bi_2100.csv")

Integrating 1 scenarios, 1 gas cycle parameter sets, 1 thermal response parameter sets, over ['carbon_dioxide'] forcing agents, between 0 and 111...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 111/111 [00:00<00:00, 5833.89 timestep/s]

URY
1
Integrating 1 scenarios, 1 gas cycle parameter sets, 1 thermal response parameter sets, over ['carbon_dioxide'] forcing agents, between 0 and 111...



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 111/111 [00:00<00:00, 4968.39 timestep/s]

USA
1
Integrating 1 scenarios, 1 gas cycle parameter sets, 1 thermal response parameter sets, over ['carbon_dioxide'] forcing agents, between 0 and 111...


...

YEM
84
Integrating 1 scenarios, 1 gas cycle parameter sets, 1 thermal response parameter sets, over ['carbon_dioxide'] forcing agents, between 0 and 111...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 111/111 [00:00<00:00, 5998.42 timestep/s]


ZAF
84
Integrating 1 scenarios, 1 gas cycle parameter sets, 1 thermal response parameter sets, over ['carbon_dioxide'] forcing agents, between 0 and 111...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 111/111 [00:00<00:00, 5802.99 timestep/s]


ZMB
84


In [ ]:
pd.concat(list_dfs).to_csv("fair_temp_resp_k90_hist_biconsump_v2022_0.csv")

In [ ]:
pd.concat(list_dfs).to_csv("fair_temp_resp_k90_hist_biconsump_v2022_1.csv")

In [ ]:
pd.concat(list_dfs).to_csv("fair_temp_resp_k80_hist_bi_v2022_0.csv")

In [ ]:
pd.concat(list_dfs).to_csv("fair_temp_resp_k90_hist_bi_v2022_0.csv")

In [ ]:
pd.concat(list_dfs).to_csv("fair_temp_resp_k80_hist_bi_v2022_1.csv")

In [ ]:
pd.concat(list_dfs).to_csv("fair_temp_resp_k90_hist_bi_v2022_1.csv")

In [25]:
list_dfs = []
for m in range(1,len(df)): 
  #  for i in master_array[9]:
        # shut off emissions from country i
#        data_iso = data1[data1["ISO3"] == i]
 #       data_iso = data_iso[["Year", "Total"]]
  #      data_iso.rename(columns = {'Total':'Total_iso'}, inplace = True)
   #     data_new = pd.merge(data, data_iso, on = "Year")
    #    data_new["Total"] = data_new["Total"] - data_new["Total_iso"]
    data_new = data
    data_new = data_new[["Year", "Total"]]
    data_new.rename(columns = {'Total':'carbon_dioxide'}, inplace = True) #change name to match emissions data
    data_new.rename(columns = {'Year':'year'}, inplace = True) #change name to match emissions data
    data_new["year"] = pd.to_numeric(data_new["year"]) 
    # since the carbon emissions are on MtCO2, we need to convert to GtC
    data_new['carbon_dioxide'] = (data_new['carbon_dioxide'] / 1000) / 3.67 
    data_new = data_new[data_new["year"] >= 1980].reset_index() # only keep 1980 and forward
    data_new['year'] = data_new['year'] - (min(data_new['year']) - 1) 
    #feed updated emissions to test_emissions
    test_emissions[("Test", "carbon_dioxide")] = data_new['carbon_dioxide']

        
        # therefore use central estimates for d1-3, q1; and set q2, q3 to return "correct" TCR/ECS values
    df1 = df[df['idnum'] == m] 
    test_gas_parameters[("default", "carbon_dioxide")][7] = list(df1["taunum"])[0]
    d1 = 0.903
    d2 = list(df1["d2num"])[0]
    d3 = 355
    q1 = 0.180
    rwf = list(df1["rwfnum"])[0]

    TCR = list(df1["tcrnum"])[0]
    ECS = TCR / rwf
    F_2x = 3.759

    v1 = (1-(d1/69.66) * (1-np.exp(-69.66/d1)) )
    v2 = (1-(d2/69.66) * (1-np.exp(-69.66/d2)) )
    v3 = (1-(d3/69.66) * (1-np.exp(-69.66/d3)) )

    q3 = (((TCR/F_2x) - q1*(v1-v2) - (ECS/F_2x)*v2) / (v3-v2))
    q2 = (ECS/F_2x - q1 -  q3)

    test_thermal_parameters["default",2][1] = q2
    test_thermal_parameters["default",3][1] = q3

            #test_thermal_parameters = pd.DataFrame([[d1,d2,d3],[q1,q2,q3]],
             #                                      index=['d','q'],
              #                                     columns=pd.MultiIndex.from_product([['default'],
                      #                                                                 [1,2,3]]))
    pulse_run = run_FaIR(emissions_in=test_emissions,
                         forcing_in=test_forcing,
                         gas_parameters=test_gas_parameters,
                         thermal_parameters=test_thermal_parameters)
    temp_response = pulse_run['T']
        #(temp_response.temp_response).apply(lambda x: float(x))   
            #temp_response["temp_response"] = float(temp_response["temp_response"]) 
            #temp_response.to_csv('temp_response/tempresponse' + "_" + str(k) + "_" + str(iso) + '.csv') 
    temp_response["loop"] = str(m) + "_loop_all" 
    list_dfs.append(temp_response)
pd.concat(list_dfs).to_csv(path + "/" + today + "/fair_temp_resp_k90_hist_bi_2100full.csv")

Integrating 1 scenarios, 1 gas cycle parameter sets, 1 thermal response parameter sets, over ['carbon_dioxide'] forcing agents, between 0 and 111...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 111/111 [00:00<00:00, 4544.87 timestep/s]

Integrating 1 scenarios, 1 gas cycle parameter sets, 1 thermal response parameter sets, over ['carbon_dioxide'] forcing agents, between 0 and 111...



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 111/111 [00:00<00:00, 5548.95 timestep/s]

Integrating 1 scenarios, 1 gas cycle parameter sets, 1 thermal response parameter sets, over ['carbon_dioxide'] forcing agents, between 0 and 111...


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 111/111 [00:00<00:00, 5817.63 timestep/s]


Integrating 1 scenarios, 1 gas cycle parameter sets, 1 thermal response parameter sets, over ['carbon_dioxide'] forcing agents, between 0 and 111...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 111/111 [00:00<00:00, 5903.35 timestep/s]


Integrating 1 scenarios, 1 gas cycle parameter sets, 1 thermal response parameter sets, over ['carbon_dioxide'] forcing agents, between 0 and 111...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 111/111 [00:00<00:00, 5673.09 timestep/s]


In [ ]:
pd.concat(list_dfs).to_csv("fair_temp_resp_k90_hist_biconsump_v2022_full.csv")

In [ ]:
pd.concat(list_dfs).to_csv("fair_temp_resp_k80_hist_bi_v2022_full.csv")

In [ ]:
pd.concat(list_dfs).to_csv("fair_temp_resp_k90_hist_bi_v2022_full.csv")